In [ ]:
from controller import Imager
import time
import cv2
import polanalyser as pa
from PIL import Image
import matplotlib.pyplot as plt
from mpldatacursor import datacursor
import numpy as np
import os

In [ ]:
imager = Imager()
num_cams = len(imager.cams)

exposure=30000 # set exposure time in micro-seconds

imager.cams[-1].remote_device.node_map.ExposureAuto.value = 'Off'
PIXELFORMATS = {'8':'Mono8','12':'Mono12Packed'}

PIXELFORMAT = PIXELFORMATS['8']  # set pixel format
imager.update_params(ExposureTime=exposure, PixelFormat=PIXELFORMAT)
imager._start_acquisitions()

In [ ]:
init_image = imager.capture_sequence(1, 0,0)
images = np.squeeze(np.array(init_image[0]))
im = np.squeeze(np.array(images))
plt.imshow(im, cmap=plt.get_cmap('gray'), vmin=0, vmax=256)
cbar = plt.colorbar(mappable=plt.cm.ScalarMappable(cmap=plt.get_cmap('gray')))
plt.show()
plt.hist(im.flatten(),256)
plt.show()

In [ ]:
time.sleep(10)  # Sleep time before starting a sequence
num_images = 5  # Number of images you want to capture

break_sets = 0 # In seconds
               # if == 0: 3 images are taken with no break (for repetition),
               # then then there is a break_sets.
               # if break_sets == break_images,
               # there is no repetition.
break_images = 0 #

raw_images, metadata = imager.capture_sequence(num_images, break_sets,break_images)

In [ ]:
experiment= 'exp'

if not os.path.exists(experiment):
    os.mkdir(experiment)

short = 'E'
repeat = 1
directory = f"./{experiment}/{short}{repeat}_{exposure}"
if not os.path.exists(directory):
    os.mkdir(directory)
for i, nagla in enumerate(raw_images):
    if not os.path.exists(f"./{directory}/{i}"):
        os.mkdir(f"./{directory}/{i}")
    for img, met in zip(nagla, metadata[i*num_cams:num_cams*(i+1)]):
        # To save as numpy array:
        #np.save(f'./{directory}/{i}/{met[1].strftime("%d_%H_%M_%S_%f")}_{met[2]}', img)
        # To save as png (for other format, switch '.png' to requested format):
        I = np.uint8(img) # convert it to uint8
        cv2.imwrite(f'./{directory}/{i}/{met[1].strftime("%d_%H_%M_%S_%f")}_{met[2]}'+'.png',I)

In [ ]:
imager._stop_acquisitions()


